In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
# Input Parameters

In [16]:
W=50
L=50
H=50
Q=3
M=1
E = (H*L)*0.05
potts_info=pd.DataFrame.as_matrix(pd.read_csv("potts_info.tab",sep="\t"))
Theta_start=np.array([[0.000000,1.1253106,1.0484147],[1.125311,0.0000000,0.1937998],[1.048415,0.1937998,0.0000000]])
Theta_0_start=np.array([[0.0000000,-1.1175560,-0.3281917],[-1.1175560,0.0000000,0.4861686],[-0.3281917,0.4861686,0.0000000]])
Delta_start=np.zeros( (50,50) )
P_start=pd.DataFrame.as_matrix(pd.read_csv("pstart.tab",sep="\t",header=None))
d_start=1.418896
d=1.418996
mu=0.5
mu_0=-0.5
sigma=1
sigma_0=1
e=-2.94
f=1.4
dd=-2.94
ff=1.4
a_d=0.001
b_d=0.001
P=P_start
P_temp=np.zeros( (50,50) )
Theta_temp=np.zeros( (50,50) )
Delta_temp=np.zeros( (50,50) )
Delta_0_temp=np.zeros( (50,50) )

C:\Users\svish\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\Users\svish\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [17]:
# Functions

In [18]:
def getIndex(potts_info, h, l): 
    index = 0
    while(h != potts_info[index, 0] or l != potts_info[index, 1]):
         index = index + 1
    return index + 1

In [35]:
def hamiltonian2_c(P, Theta, Delta):
    hamiltonian = 0
    L = P.shape[0]
    H = P.shape[1]
    for h in range(0,H-1):
        for l in range(0,L-1):
            if P[h, l] != 0 and Delta[h, l] != 0:
                if Delta[h, l + 1] != 0 and P[h, l] != P[h, l + 1]:
                    try:
                        hamiltonian = hamiltonian + Theta[int(P[h, l]) - 1, int(P[h, l + 1] )- 1]
                    except IndexError:
                        pass
                if Delta[h + 1, l] != 0 and P[h, l] != P[h + 1, l]:
                    try:
                        hamiltonian = hamiltonian + Theta[int(P[h + 1, l]) - 1, int(P[h, l]) - 1]
                    except IndexError:
                        pass


    h = H - 1
    for l in range(0,L-1):
        if P[h, l] != 0 and Delta[h, l] != 0 and Delta[h, l + 1] != 0 and P[h, l] != P[h, l + 1]:
            try :
                hamiltonian = hamiltonian + Theta[int(P[h, l]) - 1, int(P[h, l + 1]) - 1]
            except:
                IndexError
    l = L - 1
    for h in range(0,H-1):
        if P[h, l] != 0 and Delta[h, l] != 0 and Delta[h + 1, l] !=0 and P[h, l] != P[h + 1, l]:
            try :
                hamiltonian = hamiltonian + Theta[int(P[h, l]) - 1, int(P[h + 1, l]) - 1]
            except:
                 IndexError
    return hamiltonian

In [20]:
def potts2_c(P, Theta, Delta): 
    L = P.shape[1]
    H = P.shape[0]
    Q = Theta.shape[0]
    prob_temp=[0]*50
    state=[0]*50
    temp = 0
    for q in range(0,Q):
        state[q] = q
    for h in range(0,H):
        for l in range(0,L):
            if P[h, l] != 0 and Delta[h, l] != 0:
                for q in range(0,Q):
                    prob_temp[q] = 0
                    if l < L - 1 and Delta[h, l + 1] != 0 and (P[h, l + 1] - 1) != q:
                        prob_temp[int(q)] = prob_temp[int(q)] + Theta[int(P[h, l + 1]) - 1, q]

                    if l > 0 and Delta[h, l - 1] != 0 and (P[h, l - 1] - 1) != q:
                        prob_temp[int(q)] = prob_temp[int(q)] + Theta[int(P[h, l - 1]) - 1, q]

                    if h < H - 1 and Delta[h + 1, l] != 0 and (P[h + 1, l] - 1) != q:
                        prob_temp[int(q)] = prob_temp[int(q)] + Theta[int(P[h + 1, l]) - 1, q]

                    if h > 0 and Delta[h - 1, l] != 0 and (P[h - 1, l] - 1) != q:
                        prob_temp[int(q)] = prob_temp[int(q)] + Theta[int(P[h - 1, l]) - 1, q]

                    prob_temp[int(q)] = np.exp(prob_temp[int(q)])

                temp = 0
                for q in range(0,Q):
                    temp = temp + prob_temp[q]
                for q in range(0,Q):
                    prob_temp[q] = prob_temp[q]/temp

                P[h, l] = np.random.choice(state,1,True,prob_temp)+1
    return P

In [21]:
# Declare variables 

In [22]:
Theta=np.zeros((3,3))
Theta_0=np.zeros((3,3))
state=[0]*50
Delta=np.zeros((50,50))
Delta_0=np.zeros((50,50))
Delta_ppi=np.zeros((50,50))
P_1=np.zeros((50,50))
P_2=np.zeros((50,50))
P_3=np.zeros((50,50))
P_4=np.zeros((50,50))
phi = 0.1
hastings = 0
temp_probs=[0] * 50
temp_count=[0]*50
d_store=[]*10
P=P_start
accept=[0,0,0,0]
burn=2
tau = 0.1
iteration=10
d_store=[0]*iterations
theta_store= np.zeros((iterations, int(Q*(Q - 1)/2)))
theta_0_store= np.zeros((iterations, int(Q*(Q - 1)/2)))
Delta_sum=[0]*iterations

In [23]:


# Initialization

In [24]:
for q in range(0,3):
    state[q]=q
    for qq in range(0,3):
        Theta[q,qq]=Theta_start[q,qq]
        Theta_0[q,qq]=Theta_0_start[q,qq]
for h in range(0,50):
    for l in range(0,50):
        Delta[h,l]=Delta_start[h,l]
        Delta_0[h,l]=1-Delta[h,l]
        Delta_ppi[h,l]=0
        P_1[h,l]=0
        P_2[h,l]=0
        P_3[h,l]=0
        P_4[h,l]=0
        temp_sum=0
        for q in range(0,3):
            temp_sum = temp_sum + potts_info[getIndex(potts_info,h + 1, l + 1)- 1, q + 2]
            temp_sum = temp_sum + potts_info[getIndex(potts_info, max(1, h), max(1, l)) - 1, q + 2]
            temp_sum = temp_sum + potts_info[getIndex(potts_info, max(1, h), l + 1) - 1, q + 2]
            temp_sum = temp_sum + potts_info[getIndex(potts_info, h + 1, max(1, l)) - 1, q + 2]
        if temp_sum==0:
            P_start[h,l]=0
            Delta[h,l]=0
            Delta_0[h,l]=0

In [25]:
# MCMC

In [38]:
    #####Update d
for iterationn in range(1,iteration):
        d_temp=np.random.gamma(d*d/phi, phi/d,1)
        hastings = (a_d - 1)*(np.log(d_temp) - np.log(d)) - b_d*(d_temp - d)
        for h in range(0,50):
            for l in range(0,50):
                if P_start[h,l]!=0:
                    # First Condition
                    if h != H - 1 and l != L - 1:
                        for qq in range(0,3):
                            temp_count[qq]=0
                        temp_count[P_start[h, l] - 1] = temp_count[P[h, l] - 1] + 1
                        if P_start[h + 1, l] != 0:
                            temp_count[P_start[h + 1, l] - 1] = temp_count[P_start[h + 1, l] - 1] + 1
                        if P_start[h, l + 1] != 0:
                            temp_count[P_start[h, l + 1] - 1] = temp_count[P_start[h, l + 1] - 1] + 1
                        if P_start[h + 1, l + 1] != 0:
                            temp_count[P_start[h + 1, l + 1] - 1] = temp_count[P_start[h + 1, l + 1] - 1] + 1
                        temp_sum = 0
                        for qq in range(0,3):
                            temp_sum = temp_sum + np.exp(d_temp*temp_count[qq])
                        for qq in range(0,3):
                            hastings = hastings + potts_info[getIndex(potts_info, h + 1, l + 1) - 1, qq + 2]*(d_temp*temp_count[qq] - np.log(temp_sum))
                        temp_sum = 0
                        for qq in range(0,3):
                            temp_sum = temp_sum + np.exp(d*temp_count[qq])
                        for qq in range(0,3):
                            hastings = hastings - potts_info[getIndex(potts_info, h + 1, l + 1) - 1, qq + 2]*(d*temp_count[qq] -np.log(temp_sum))
                    # Second Condition
                    if h != 0 and l != 0:
                        for qq in range(0,3):
                            temp_count[qq]=0
                        temp_count[P_start[h, l] - 1] = temp_count[P[h, l] - 1] + 1
                        if P_start[h - 1, l] != 0:
                            temp_count[P_start[h - 1, l] - 1] = temp_count[P_start[h - 1, l] - 1] + 1
                        if P_start[h, l - 1] != 0:
                            temp_count[P_start[h, l - 1] - 1] = temp_count[P_start[h, l - 1] - 1] + 1
                        if P_start[h - 1, l - 1] != 0:
                            temp_count[P_start[h -1, l - 1] - 1] = temp_count[P_start[h - 1, l - 1] - 1] + 1
                        temp_sum = 0
                        for qq in range(0,3):
                            temp_sum = temp_sum + np.exp(d_temp*temp_count[qq])
                        for qq in range(0,3):
                            hastings = hastings + potts_info[getIndex(potts_info, h , l ) - 1, qq + 2]*(d_temp*temp_count[qq] - np.log(temp_sum))
                        temp_sum = 0
                        for qq in range(0,3):
                            temp_sum = temp_sum + np.exp(d*temp_count[qq])
                        for qq in range(0,3):
                            hastings = hastings - potts_info[getIndex(potts_info, h , l ) - 1, qq + 2]*(d*temp_count[qq] -np.log(temp_sum))
                    # Third Condition
                    if h != H-1 and l != 0:
                        for qq in range(0,3):
                            temp_count[qq]=0
                        temp_count[P_start[h, l] - 1] = temp_count[P[h, l] - 1] + 1
                        if P_start[h +1, l] != 0:
                            temp_count[P_start[h + 1, l] - 1] = temp_count[P_start[h +1, l] - 1] + 1
                        if P_start[h, l - 1] != 0:
                            temp_count[P_start[h, l - 1] - 1] = temp_count[P_start[h, l - 1] - 1] + 1
                        if P_start[h + 1, l - 1] != 0:
                            temp_count[P_start[h +1, l - 1] - 1] = temp_count[P_start[h + 1, l - 1] - 1] + 1
                        temp_sum = 0
                        for qq in range(0,3):
                            temp_sum = temp_sum + np.exp(d_temp*temp_count[qq])
                        for qq in range(0,3):
                            hastings = hastings + potts_info[getIndex(potts_info, h + 1, l ) - 1, qq + 2]*(d_temp*temp_count[qq] - np.log(temp_sum))
                        temp_sum = 0
                        for qq in range(0,3):
                            temp_sum = temp_sum + np.exp(d*temp_count[qq])
                        for qq in range(0,3):
                            hastings = hastings - potts_info[getIndex(potts_info, h + 1, l ) - 1, qq + 2]*(d*temp_count[qq] -np.log(temp_sum))

                    # Fourth Condition 
                    if h != 0 and l != L-1:
                        for qq in range(0,3):
                            temp_count[qq]=0
                        temp_count[P_start[h, l] - 1] = temp_count[P[h, l] - 1] + 1
                        if P_start[h -1, l] != 0:
                            temp_count[P_start[h - 1, l] - 1] = temp_count[P_start[h -1, l] - 1] + 1
                        if P_start[h, l +1] != 0:
                            temp_count[P_start[h, l +1] - 1] = temp_count[P_start[h, l + 1] - 1] + 1
                        if P_start[h - 1, l + 1] != 0:
                            temp_count[P_start[h -1, l +1] - 1] = temp_count[P_start[h - 1, l + 1] - 1] + 1
                        temp_sum = 0
                        for qq in range(0,3):
                            temp_sum = temp_sum + np.exp(d_temp*temp_count[qq])
                        for qq in range(0,3):
                            hastings = hastings + potts_info[getIndex(potts_info, h, l+1 ) - 1, qq + 2]*(d_temp*temp_count[qq] - np.log(temp_sum))
                        temp_sum = 0
                        for qq in range(0,3):
                            temp_sum = temp_sum + np.exp(d*temp_count[qq])
                        for qq in range(0,3):
                            hastings = hastings - potts_info[getIndex(potts_info, h , l+1 ) - 1, qq + 2]*(d*temp_count[qq] -np.log(temp_sum))
        if hastings >= np.log(random.randint(0,10001)/10000):
            d = d_temp
            if f > burn:
                accept[3]=accept[3]+1
        ###### update P
        for e in range(0,int(E)):
            h=random.randint(0,H-1)
            l=random.randint(0,L-1)
            if P[h,l]!=0:
                for hh in range(0,H):
                    for ll in range(0,L):
                        P_temp[hh, ll] = P[hh, ll]
                for q in range(0,Q):
                    P_temp[h, l] = q + 1
                if Delta[h, l] == 1:
                    temp_probs[q] = hamiltonian2_c(P_temp, Theta, Delta)
                #else:
                    #temp_probs[q] = hamiltonian2_c(P_temp, Theta_0, Delta_0)
                #1
                if h != H - 1 and l != L - 1:
                    for qq in range(0,Q):
                        temp_count[qq] = 0
                    temp_count[int(P_temp[h, l]) - 1] = temp_count[int(P_temp[h, l]) - 1] + 1
                    if P_temp[h + 1, l] != 0:
                        temp_count[int(P_temp[h + 1, l]) - 1] = temp_count[int(P_temp[h + 1, l]) - 1] + 1
                    if P_temp[h, l + 1]!= 0:
                        temp_count[int(P_temp[h, l + 1]) - 1] = temp_count[int(P_temp[h, l + 1]) - 1] + 1
                    if P_temp[h + 1, l + 1] != 0:
                        temp_count[int(P_temp[h + 1, l + 1]) - 1] = temp_count[int(P_temp[h + 1, l + 1]) - 1] + 1
                    temp_sum = 0
                    for qq in range(0,Q):
                        temp_sum = temp_sum + np.exp(d*temp_count[qq])
                    for qq in range(0,Q):
                        temp_probs[q] = temp_probs[q] + potts_info[getIndex(potts_info, h + 1, l + 1) - 1, qq + 2]*(d*temp_count[qq] - np.log(temp_sum))
                #2
                if h != 0 and l != 0:
                    for qq in range(0,Q):
                        temp_count[qq] = 0
                    temp_count[int(P_temp[h, l]) - 1] = temp_count[int(P_temp[h, l]) - 1] + 1
                    if P_temp[h -1, l] != 0:
                        temp_count[int(P_temp[h - 1, l]) - 1] = temp_count[int(P_temp[h - 1, l]) - 1] + 1
                    if P_temp[h, l - 1]!= 0:
                        temp_count[int(P_temp[h, l - 1]) - 1] = temp_count[int(P_temp[h, l - 1]) - 1] + 1
                    if P_temp[h - 1, l - 1] != 0:
                        temp_count[int(P_temp[h - 1, l - 1]) - 1] = temp_count[int(P_temp[h - 1, l - 1]) - 1] + 1
                    temp_sum = 0
                    for qq in range(0,Q):
                        temp_sum = temp_sum + np.exp(d*temp_count[qq])
                    for qq in range(0,Q):
                        temp_probs[q] = temp_probs[q] + potts_info[getIndex(potts_info, h , l ) - 1, qq + 2]*(d*temp_count[qq] - np.log(temp_sum))
                #3
                if h != H-1 and l != 0:
                    for qq in range(0,Q):
                        temp_count[qq] = 0
                    temp_count[int(P_temp[h, l]) - 1] = temp_count[int(P_temp[h, l]) - 1] + 1
                    if P_temp[h +1, l] != 0:
                        temp_count[int(P_temp[h + 1, l]) - 1] = temp_count[int(P_temp[h + 1, l]) - 1] + 1
                    if P_temp[h, l - 1]!= 0:
                        temp_count[int(P_temp[h, l - 1]) - 1] = temp_count[int(P_temp[h, l - 1]) - 1] + 1
                    if P_temp[h + 1, l - 1] != 0:
                        temp_count[int(P_temp[h +1, l - 1]) - 1] = temp_count[int(P_temp[h +1, l - 1]) - 1] + 1
                    temp_sum = 0
                    for qq in range(0,Q):
                        temp_sum = temp_sum + np.exp(d*temp_count[qq])
                    for qq in range(0,Q):
                        temp_probs[q] = temp_probs[q] + potts_info[getIndex(potts_info, h+1, l ) - 1, qq + 2]*(d*temp_count[qq] - np.log(temp_sum))
                #4
                if h != 0 and l != L-1:
                    for qq in range(0,Q):
                        temp_count[qq] = 0
                    temp_count[int(P_temp[h, l]) - 1] = temp_count[int(P_temp[h, l]) - 1] + 1
                    if P_temp[h -1, l] != 0:
                        temp_count[int(P_temp[h - 1, l]) - 1] = temp_count[int(P_temp[h - 1, l]) - 1] + 1
                    if P_temp[h, l + 1]!= 0:
                        temp_count[int(P_temp[h, l + 1]) - 1] = temp_count[int(P_temp[h, l +1]) - 1] + 1
                    if P_temp[h - 1, l - 1] != 0:
                        temp_count[int(P_temp[h -1, l - 1]) - 1] = temp_count[int(P_temp[h -1, l - 1]) - 1] + 1
                    temp_sum = 0
                    for qq in range(0,Q):
                        temp_sum = temp_sum + np.exp(d*temp_count[qq])
                    for qq in range(0,Q):
                        temp_probs[q] = temp_probs[q] + potts_info[getIndex(potts_info, h, l+1 ) - 1, qq + 2]*(d*temp_count[qq] - np.log(temp_sum))
                temp_sum = 0
                temp_max = max(temp_probs)
                for q in range(0,Q):
                    temp_probs[q] = temp_probs[q] - temp_max
                    temp_probs[q] = np.exp(temp_probs[q])
                    temp_sum = temp_sum + temp_probs[q]
                for q in range(0,Q):
                    temp_probs[q] = temp_probs[q]/temp_sum
                temp=np.random.choice(state,1,True,temp_probs)+1
                P[h, l] = temp
        # Update Theta
        for q in range(0,Q):
            for qq in range(0,Q):
                for qqq in range(0,Q):
                    for qqqq in range(0,Q):
                        Theta_temp[qqq, qqqq] = Theta[qqq, qqqq]


            Theta_temp[q, qq] = np.random.normal(Theta[q, qq], tau, 1)
            Theta_temp[qq, q] = Theta_temp[q, qq]
            for h in range(0,H):
                for l in range(0,L):
                    P_temp[h, l] = P[h, l]

            for m in range(0,M):
                P_temp = potts2_c(P_temp, Theta_temp, Delta)
            hastings = hamiltonian2_c(P_temp, Theta, Delta) - hamiltonian2_c(P, Theta, Delta) + hamiltonian2_c(P, Theta_temp, Delta) - hamiltonian2_c(P_temp, Theta_temp, Delta)
            hastings = hastings - np.log(sigma) - (Theta_temp[q, qq] - mu)*(Theta_temp[q, qq] - mu)/2/sigma/sigma

            if hastings >= np.log(random.randint(0,10001)/10000):
                Theta[q, qq] = Theta_temp[q, qq]
                Theta[qq, q] = Theta[q, qq]
                if f > burn:
                    accept[0]=accept[0]+1
        # Update Theta_0
        for q in range(0,Q):
            for qq in range(0,Q):
                for qqq in range(0,Q):
                    for qqqq in range(0,Q):
                        Theta_temp[qqq, qqqq] = Theta_0[qqq, qqqq]


            Theta_temp[q, qq] = np.random.normal(Theta_0[q, qq], tau, 1)
            Theta_temp[qq, q] = Theta_temp[q, qq]
            for h in range(0,H):
                for l in range(0,L):
                    P_temp[h, l] = P[h, l]

            for m in range(0,M):
                P_temp = potts2_c(P_temp, Theta_temp, Delta)
            hastings = hamiltonian2_c(P_temp, Theta, Delta) - hamiltonian2_c(P, Theta, Delta) + hamiltonian2_c(P, Theta_temp, Delta) - hamiltonian2_c(P_temp, Theta_temp, Delta)
            hastings = hastings - np.log(sigma) - (Theta_temp[q, qq] - mu)*(Theta_temp[q, qq] - mu)/2/sigma/sigma

            if hastings >= np.log(random.randint(0,10001)/10000):
                Theta[q, qq] = Theta_temp[q, qq]
                Theta[qq, q] = Theta[q, qq]
                if f > burn:
                    accept[0]=accept[0]+1
         # Update Delta
        for e in range(0,int(E)):
            h=random.randint(0,H-1)
            l=random.randint(0,L-1)
            if P[h,l]!=0:
                for hh in range(0,H):
                    for ll in range(0,L):
                        Delta_temp[hh, ll] = Delta[hh, ll]
                        Delta_0_temp[hh, ll] = 1-Delta[hh, ll]
                        P_temp[hh, ll] = P[hh, ll]
                Delta_temp[hh, ll] = 1-Delta[hh, ll]   
                Delta_0_temp[hh, ll] = 1-Delta_0_temp[hh, ll]
                for m in range(0,M):
                    P_temp = potts2_c(P_temp, Theta, Delta_temp)
                    P_temp = potts2_c(P_temp, Theta_0, Delta_0_temp)
                hastings = hamiltonian2_c(P_temp, Theta, Delta) - hamiltonian2_c(P, Theta, Delta) + hamiltonian2_c(P, Theta, Delta_temp) - hamiltonian2_c(P_temp, Theta, Delta_temp)
                hastings = hastings + hamiltonian2_c(P_temp, Theta_0, Delta_0) - hamiltonian2_c(P, Theta_0, Delta_0) + hamiltonian2_c(P, Theta_0, Delta_0_temp) - hamiltonian2_c(P_temp, Theta_0, Delta_0_temp)
                hh=h
                ll=l
                if Delta[hh, ll] == 0:
                    hastings = hastings + (dd + ff*(Delta[min(hh + 1, H - 1), ll] + Delta[max(hh - 1, 0), ll] + Delta[hh, min(ll + 1, L - 1)] + Delta[hh, max(ll - 1, 0)]))
                else :
                    Delta[hh, ll] = 0
                    hastings = hastings - (dd + ff*(Delta[min(hh + 1, H - 1), ll] + Delta[max(hh - 1, 0), ll] + Delta[hh, min(ll + 1, L - 1)] + Delta[hh, max(ll - 1, 0)]))
                    Delta[hh, ll] = 1
            #1
            if h != H - 1:
                hh = h + 1
                ll = l
                temp = Delta[hh, ll]
                temp_2 = Delta_temp[hh, ll]
                Delta[hh, ll] = 0
                Delta_temp[hh, ll] = 0
                hastings = hastings + np.log(1 + np.exp(dd + ff*(Delta[min(hh + 1, H - 1), ll] + Delta[max(hh - 1, 0), ll] + Delta[hh, min(ll + 1, L - 1)] + Delta[hh, max(ll - 1, 0)]))) - np.log(1 + np.exp(dd + ff*(Delta_temp[min(hh + 1, H - 1), ll] + Delta_temp[max(hh - 1, 0), ll] + Delta_temp[hh, min(ll + 1, L - 1)] + Delta_temp[hh, max(ll - 1, 0)])))
                Delta[hh, ll] = temp
                Delta_temp[hh, ll] = temp_2
                if Delta[hh, ll] == 1:
                    if(Delta[h, l] == 0): #Add
                        hastings = hastings + ff
                    else: #Delete
                        hastings = hastings - ff
            #2
            if h != 0:
                hh = h - 1
                ll = l
                temp = Delta[hh, ll]
                temp_2 = Delta_temp[hh, ll]
                Delta[hh, ll] = 0
                Delta_temp[hh, ll] = 0
                hastings = hastings + np.log(1 + np.exp(dd + ff*(Delta[min(hh + 1, H - 1), ll] + Delta[max(hh - 1, 0), ll] + Delta[hh, min(ll + 1, L - 1)] + Delta[hh, max(ll - 1, 0)]))) - np.log(1 + np.exp(dd + ff*(Delta_temp[min(hh + 1, H - 1), ll] + Delta_temp[max(hh - 1, 0), ll] + Delta_temp[hh, min(ll + 1, L - 1)] + Delta_temp[hh, max(ll - 1, 0)])))
                Delta[hh, ll] = temp
                Delta_temp[hh, ll] = temp_2
                if Delta[hh, ll] == 1:
                    if(Delta[h, l] == 0) :#Add
                        hastings = hastings + ff
                    else :#Delete
                        hastings = hastings - ff
             #3
            if l != L-1:
                hh = h 
                ll = l+1
                temp = Delta[hh, ll]
                temp_2 = Delta_temp[hh, ll]
                Delta[hh, ll] = 0
                Delta_temp[hh, ll] = 0
                hastings = hastings + np.log(1 + np.exp(dd + ff*(Delta[min(hh + 1, H - 1), ll] + Delta[max(hh - 1, 0), ll] + Delta[hh, min(ll + 1, L - 1)] + Delta[hh, max(ll - 1, 0)]))) - np.log(1 + np.exp(dd + ff*(Delta_temp[min(hh + 1, H - 1), ll] + Delta_temp[max(hh - 1, 0), ll] + Delta_temp[hh, min(ll + 1, L - 1)] + Delta_temp[hh, max(ll - 1, 0)])))
                Delta[hh, ll] = temp
                Delta_temp[hh, ll] = temp_2
                if Delta[hh, ll] == 1:
                    if(Delta[h, l] == 0): #Add
                        hastings = hastings + ff
                    else: #Delete
                        hastings = hastings - ff
           #4
            if l != 0:
                hh = h 
                ll = l-1
                temp = Delta[hh, ll]
                temp_2 = Delta_temp[hh, ll]
                Delta[hh, ll] = 0
                Delta_temp[hh, ll] = 0
                hastings = hastings + np.log(1 + np.exp(dd + ff*(Delta[min(hh + 1, H - 1), ll] + Delta[max(hh - 1, 0), ll] + Delta[hh, min(ll + 1, L - 1)] + Delta[hh, max(ll - 1, 0)]))) - np.log(1 + np.exp(dd + ff*(Delta_temp[min(hh + 1, H - 1), ll] + Delta_temp[max(hh - 1, 0), ll] + Delta_temp[hh, min(ll + 1, L - 1)] + Delta_temp[hh, max(ll - 1, 0)])))
                Delta[hh, ll] = temp
                Delta_temp[hh, ll] = temp_2
                if Delta[hh, ll] == 1:
                    if(Delta[h, l] == 0): #Add
                        hastings = hastings + ff
                else :#Delete
                    hastings = hastings - ff
            if hastings >= np.log(random.randint(0,10001)/10000):
                Delta[h, l] = 1-Delta[h,l]
                Delta_0[h, l] = 1-Delta[h,l]
                if f > burn:
                    accept[2]=accept[2]+1
    print(" %d, Iteration number :" %(f)) 
    d_store[iterationn] = d
    for q in range(0,3):
        for qq in range(0,3):
            theta_store[iterationn, 0] = Theta[q, qq]
            theta_0_store[iterationn, 0] = Theta_0[q, qq]
            theta_store[iterationn, 1] = Theta[q, qq]
            theta_0_store[iterationn, 1] = Theta_0[q, qq]
            theta_store[iterationn, 2] = Theta[q, qq]
            theta_0_store[iterationn, 2] = Theta_0[q, qq]
            temp=0
    for h in range(0,H):
        for l in range(0,L):
            if iteration> burn:
                Delta_ppi[h, l] = Delta_ppi[h, l] + Delta[h, l]
                if P[h, l] == 1:
                    P_1[h, l] = P_1[h, l] + 1
                if P[h, l] == 2:
                    P_2[h, l] = P_2[h, l] + 1
                if P[h, l] == 3:
                    P_3[h, l] = P_3[h, l] + 1
                if P[h, l] == 4:
                    P_4[h, l] = P_4[h, l] + 1
                temp = temp + Delta[h, l]
                Delta[iteration]=temp
    for h in range(0,H):
        for l in range(0,L):
            Delta_ppi[h, l] = Delta_ppi[h, l]/(iterationn - burn)
    #accept[0] = accept[0]/(iter - burn)/(Q*(Q - 1)/2)
   # accept[1] = accept[1]/(iter - burn)/(Q*(Q - 1)/2)
    #accept[2] = accept[2]/(iter - burn)/E
    #accept[3] = accept[3]/(iter - burn)

C:\Users\svish\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\svish\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
C:\Users\svish\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in greater_equal
C:\Users\svish\Anaconda3\lib\site-packages\ipykernel_launcher.py:282: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\svish\Anaconda3\lib\site-packages\ipykernel_launcher.py:298: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\svish\Anaconda3\lib\site-packages\ipykernel_launcher.py:266: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\svish\Anaconda3\lib\site-packages\ipykernel_launcher.py:314: RuntimeWarning: invalid value encountered in d

 1, Iteration number :


In [39]:
print(Theta)

[[0.         1.1253106  0.78572969]
 [1.125311   0.         1.50369069]
 [0.78572969 1.50369069 0.18423974]]


In [40]:
print(Theta_0)

[[ 0.        -1.117556  -0.3281917]
 [-1.117556   0.         0.4861686]
 [-0.3281917  0.4861686  0.       ]]


In [44]:
Delta_ppi

array([[0.14285714, 0.14285714, 0.        , ..., 0.16326531, 0.        ,
        0.14285714],
       [0.        , 0.14285714, 0.02040816, ..., 0.16326531, 0.        ,
        0.        ],
       [0.        , 0.16326531, 0.14285714, ..., 0.02040816, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.16326531, ..., 0.16326531, 0.        ,
        0.14285714],
       [0.14285714, 0.02040816, 0.14285714, ..., 0.14285714, 0.16326531,
        0.        ],
       [0.        , 0.        , 0.14285714, ..., 0.16326531, 0.16326531,
        0.16326531]])